In [17]:
import sys
sys.path.append('../quackd')

import qiskit
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute
from quantuminspire.qiskit import QI
from qiskit.ignis.mitigation.measurement import complete_meas_cal, CompleteMeasFitter

In [18]:
import quackd.utils as u
u.set_qi_auth("../credentials/auth.json")

In [56]:
def create_calibration_matrix(qubits, qubit_list):
        """
        qubits [int]: number of qubits
        qubit_list [list] = list/array of qubits indexed into the backend
        """
        # create calibration circuits
        qr = QuantumRegister(qubits)
        meas_calibs, state_labels = complete_meas_cal(qubit_list=qubit_list, qr=qr, circlabel='cal')
        # create calibration matrix by running calibration
        backend = QI.get_backend('QX single-node simulator') # replace 'with Starmon-5' later
        job = qiskit.execute(meas_calibs, backend=backend, shots=1024)
        cal_results = job.result()
        meas_fitter = CompleteMeasFitter(cal_results, state_labels, circlabel='cal')
        return meas_fitter

In [57]:
def apply_measurement_error_mitigation(meas_fitter, raw_counts):
        """
        meas_fitter [qiskit ignis CompleteMeasFitter object]: calibration matrix fitter
        raw_counts [dict]: dictionary of bitstrings with respective counts
        """
        # apply measurement error mitigation via calibration matrix ####
        meas_filter = meas_fitter.filter
        mitigated_results = meas_filter.apply(raw_counts)
        return mitigated_results

In [ ]:
# Example

In [61]:
# raw results
qc = QuantumCircuit(1,1)
qc.measure(0,0)
qc.draw()
test_job = qiskit.execute(qc, backend=QI.get_backend('QX single-node simulator'), shots=1024)
test_counts = test_job.result().get_counts()
test_counts 

{'0': 1024.0}

In [62]:
# mitigated results (same as raw results if there is no noise)
example_meas_fitter = create_calibration_matrix(1, [0])
apply_measurement_error_mitigation(example_meas_fitter, test_counts)

{'0': 1023.9999999079325, '1': 9.206719772669736e-08}